In [1]:
import pandas as pd
import numpy as np
import os

from itertools import product

In [2]:
def trata_dados(df, ano, pop_name):
    df['ID_Municipio'] = df['Município'].str[:6] #coluna do codigo do municipio
    df['Município'] = df['Município'].str[7:] #coluna do nome do municipio
    df['ano'] = ano #coluna do ano dos dados

    df.columns = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total', 'ID_Municipio', 'ano']
        
    df['ID_Municipio'] = pd.to_numeric(df['ID_Municipio'])
        
    #nao sei explicar direito o que faz, mas deixa tudo no formato desejado
    df = df.melt(id_vars=['Município', 'ID_Municipio', 'ano'], var_name='faixa_etaria', value_name=pop_name)
    
    return df

In [3]:
def gerar_df(cols, csv_name, value_name):
    df = pd.read_csv(f"dados populacionais/{csv_name}.csv", encoding='latin1', sep=';')

    padrao = df.iloc[[-1]]
    padrao.columns = cols

    padrao = padrao.melt(id_vars=['Município'], var_name='faixa_etaria', value_name=value_name)
    padrao.drop(labels=['Município'], axis=1, inplace=True)
    
    return df, padrao

In [4]:
#gera o df com a populacao total padrao de 2010

cols = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total']

#gera o df com a populacao total padrao de 2010
df, padrao_t = gerar_df(cols, "dados total/2010total", "pop_pad_total")

#gera o df com a populacao masculina padrao de 2010
df, padrao_h = gerar_df(cols, "dados homem/homem2010", "pop_pad_homem")

#gera o df com a populacao feminina padrao de 2010
df, padrao_m = gerar_df(cols, "dados mulher/mulher2010", "pop_pad_mulher")


In [5]:
padrao_t

,faixa_etaria,pop_pad_total
0,0a4,1338553
1,5a9,1497792
2,10a14,1711110
3,15a19,1742648
4,20a24,1757058
5,25a29,1727867
6,30a34,1617371
7,35a39,1435712
8,40a44,1392455
9,45a49,1312183


In [6]:
def gera_df_e_remove_linha(csv_name, title):
    #populacao total
    df = pd.read_csv(f"dados populacionais/{csv_name}.csv", encoding='latin1', sep=';')
    df.drop(df.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

    df = trata_dados(df, 2010, title)
    return df

#populacao total
df = gera_df_e_remove_linha("dados total/2010total", "pop_total")

#populacao masculina
df_h = gera_df_e_remove_linha("dados homem/homem2010", "pop_homem")

#populacao feminina
df_m = gera_df_e_remove_linha("dados mulher/mulher2010", "pop_mulher")


In [7]:
def pegar_arquivos(df, pasta, csv_name, title):
    directory = os.fsencode(f"./dados populacionais/{pasta}/") #populacao total

    anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

    i = 0

    #cria o dataframe com populacao total
    for file in os.listdir(directory):
        filename = os.fsdecode(file)

        if filename != f'{csv_name}.csv':
            aux = pd.read_csv(f"./dados populacionais/{pasta}/" + filename, encoding='latin1', sep=";")
            aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

            aux = trata_dados(aux, anos[i], title)

            df = df.append(aux)

            i = i + 1 #avanca o index de anos
            
    return df
            
df = pegar_arquivos(df, "dados total", "2010total", "pop_total")
df_h = pegar_arquivos(df_h, "dados homem", "homem2010", "pop_homem")
df_m = pegar_arquivos(df_m, "dados mulher", "mulher2010", "pop_mulher")

In [8]:
def unir_dfs(df1, df2, padrao2):
    df1 = df1.merge(df2, how="inner")
    df1 = df1.merge(padrao2, how="inner")
    return df1

df = df.merge(padrao_t, how="inner")

df = unir_dfs(df, df_h, padrao_h)
df = unir_dfs(df, df_m, padrao_m)

df

,Município,ID_Municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher
0,Abadia dos Dourados,310010,2010,0a4,418,1338553,211,684629,207,653924
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924
2,Abre Campo,310030,2010,0a4,914,1338553,464,684629,450,653924
3,Acaiaca,310040,2010,0a4,279,1338553,135,684629,144,653924
4,Açucena,310050,2010,0a4,687,1338553,368,684629,319,653924
...,...,...,...,...,...,...,...,...,...,...
153535,Virginópolis,317180,2019,Total,10510,19957444,5180,9828254,5330,10129190
153536,Virgolândia,317190,2019,Total,5380,19957444,2622,9828254,2758,10129190
153537,Visconde do Rio Branco,317200,2019,Total,42564,19957444,21194,9828254,21370,10129190
153538,Volta Grande,317210,2019,Total,5252,19957444,2512,9828254,2740,10129190


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153540 entries, 0 to 153539
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Município       153540 non-null  object
 1   ID_Municipio    153540 non-null  int64 
 2   ano             153540 non-null  int64 
 3   faixa_etaria    153540 non-null  object
 4   pop_total       153540 non-null  int64 
 5   pop_pad_total   153540 non-null  int64 
 6   pop_homem       153540 non-null  int64 
 7   pop_pad_homem   153540 non-null  int64 
 8   pop_mulher      153540 non-null  int64 
 9   pop_pad_mulher  153540 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 12.9+ MB


In [10]:
#usado para obter os nomes de todos os agravos e seus subgrupos
agrav = pd.read_csv("Planilha para o R - Planilha.csv")
agrav = agrav.drop(labels='Unnamed: 0', axis=1)
agrav = agrav.fillna(0)
agrav.columns = ['cidsize', 'cid', 'agravo', 'subgrupo']

#listas com o nomes das doenças, anos, e meses
nomes_agravo = np.concatenate((agrav['agravo'].unique(), agrav['subgrupo'].unique()[1:]), axis=None)
faixas = df['faixa_etaria'].unique()
faixas = faixas[:-1]
anos = [ano for ano in range(2010, 2020)]
meses = [mes for mes in range(1, 13)]

In [11]:
#grupos de cidades analisadas
cidades_selecionadas = ["Betim", 'Brumadinho','Curvelo','Esmeraldas','Florestal','Fortuna de Minas',
                        'Igarapé','Juatuba','Maravilhas','Mário Campos','Martinho Campos',
                        'Papagaios','Pará de Minas','Paraopeba','Pequi','Pompéu','São Joaquim de Bicas',
                        'São José da Varginha','Sarzedo']

cidades_limitrofes = ['Abaeté','Araçaí','Belo Horizonte','Belo Vale','Bom Despacho','Bonfim','Cachoeira da Prata',
                      'Caetanópolis','Capim Branco','Conceição do Pará','Contagem','Cordisburgo','Corinto',
                      'Dores do Indaiá','Felixlândia','Ibirité','Igaratinga','Inhaúma','Inimutaba','Itaúna',
                      'Itabirito','Itatiaiuçu','Leandro Ferreira','Mateus Leme','Matozinhos','Moeda',
                      'Morada Nova de Minas','Morro da Garça','Nova Lima','Onça de Pitangui','Paineiras',
                      'Pedro Leopoldo','Pitangui','Presidente Juscelino','Quartel Geral','Ribeirão das Neves',
                      'Rio Manso','Santana de Pirapama','Santo Hipólito','Sete Lagoas']


cidades_prox_limitrofes = ['Araújos','Augusto de Lima','Baldim','Biquinhas','Carmo do Cajuru','Cedro do Abaeté',
                           'Conceição do Mato Dentro','Confins','Congonhas','Congonhas do Norte','Crucilândia',
                           'Estrela do Indaiá','Funilândia','Gouveia','Itaguara','Jaboticatubas','Jeceaba',
                           'Jequitibá','Lagoa Santa','Lassance','Luz','Moema','Monjolos','Nova Serrana',
                           'Ouro Preto','Piedade dos Gerais','Prudente de Morais',
                           'Raposos','Rio Acima','Sabará','São Gonçalo do Abaeté','São Gonçalo do Pará',
                           'São Gotardo','São José da Lapa','Santa Bárbara','Santa Luzia','Santana do Riacho',
                           'Santo Antônio do Monte','Serra da Saudade','Tiros','Vespasiano']


#gera uma lista com o ID de todas as cidades a serem avaliadas
todas = cidades_selecionadas + cidades_limitrofes + cidades_prox_limitrofes

cidades = df[df['Município'].isin(todas)]['ID_Municipio'].unique()

In [12]:
#cria um dataframe vazio, que armazenará todas as permutações de municipio, ano, mês e agravo
df_size = len(list(product(cidades, faixas, anos, meses, nomes_agravo)))
df_index = [i for i in range(df_size)]

agregados = pd.DataFrame({'id_municipio': pd.Series([], dtype='float32'),
                            'faixa_etaria': pd.Series([], dtype='str'),
                            'ano': pd.Series([], dtype='float32'),
                            'mes': pd.Series([], dtype='float32'),
                            'agravo': pd.Series([], dtype='str')},
                            index = df_index)

agregados

,id_municipio,faixa_etaria,ano,mes,agravo
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
13871995,NaN,NaN,NaN,NaN,NaN
13871996,NaN,NaN,NaN,NaN,NaN
13871997,NaN,NaN,NaN,NaN,NaN
13871998,NaN,NaN,NaN,NaN,NaN


In [13]:
k = 0

data = agregados.values

for row in product(cidades, faixas, anos, meses, nomes_agravo):
    data[k] = row

    k = k + 1

In [14]:
np.savetxt("agregados_sih.csv", data, fmt='%s', header="ID_Municipio faixa_etaria ano mes agravo", comments='')

In [15]:
agregados = pd.read_csv("agregados_sih.csv", sep=" ")
agregados

,ID_Municipio,faixa_etaria,ano,mes,agravo
0,310020,0a4,2010,1,colera
1,310020,0a4,2010,1,diarreia
2,310020,0a4,2010,1,odip
3,310020,0a4,2010,1,tubresp
4,310020,0a4,2010,1,tbsistnerv
...,...,...,...,...,...
13871995,317120,>=80anos,2019,12,asma
13871996,317120,>=80anos,2019,12,broncoeq
13871997,317120,>=80anos,2019,12,infecpele
13871998,317120,>=80anos,2019,12,outraspele


In [16]:
sih = pd.read_csv("sih_agravos.csv", encoding="latin1")

C:\Users\Guilherme Bezerra\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,7,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
print(len(sih))
sih.drop(sih.tail(1).index,inplace=True)
print(len(sih))
sih

12099957
12099956


,IDENT,MUNIC_RES,NASC,SEXO,DT_INTER,DT_SAIDA,DIAG_PRINC,DIAG_SECUN,COD_IDADE,IDADE,...,idade2,faixaetaria,sexo2,raca2,esc,causa3,agravo,subgrupo,semana,capitulo
0,1,316240.0,19241231.0,3.0,20091221.0,20091226.0,J960,NaN,4.0,84.0,...,84.0,>=80anos,F,NaN,NaN,J96,NaN,NaN,NaN,NaN
1,1,316240.0,19930726.0,3.0,20100109.0,20100113.0,A490,NaN,4.0,16.0,...,16.0,15a19,F,NaN,1.0,A49,NaN,NaN,NaN,NaN
2,1,316240.0,19470723.0,1.0,20100108.0,20100112.0,I500,NaN,4.0,62.0,...,62.0,60a64,M,NaN,2.0,I50,NaN,NaN,NaN,NaN
3,1,316240.0,19471102.0,1.0,20100110.0,20100115.0,I500,NaN,4.0,62.0,...,62.0,60a64,M,NaN,3.0,I50,NaN,NaN,NaN,NaN
4,1,316240.0,19880313.0,1.0,20100103.0,20100107.0,J158,NaN,4.0,21.0,...,21.0,20a24,M,NaN,4.0,J15,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12099951,1,317180.0,19400904.0,3.0,20191211.0,20191214.0,J449,0.0,4.0,79.0,...,79.0,75a79,F,Parda,NaN,J44,doencviaaer,bronquite,NaN,X
12099952,1,315750.0,20030421.0,1.0,20191227.0,20191230.0,J189,0.0,4.0,16.0,...,16.0,15a19,M,Parda,NaN,J18,NaN,NaN,NaN,NaN
12099953,1,317180.0,19851212.0,1.0,20191227.0,20191231.0,K746,0.0,4.0,34.0,...,34.0,30a34,M,Parda,NaN,K74,NaN,NaN,NaN,NaN
12099954,1,317180.0,20191129.0,3.0,20191203.0,20191205.0,P599,0.0,2.0,4.0,...,0.0,0a4,F,Parda,NaN,P59,NaN,NaN,NaN,NaN


In [18]:
sih['MUNIC_RES'] = sih['MUNIC_RES'].astype(int)
sih['NASC'] = sih['NASC'].astype(int)
sih

,IDENT,MUNIC_RES,NASC,SEXO,DT_INTER,DT_SAIDA,DIAG_PRINC,DIAG_SECUN,COD_IDADE,IDADE,...,idade2,faixaetaria,sexo2,raca2,esc,causa3,agravo,subgrupo,semana,capitulo
0,1,316240,19241231,3.0,20091221.0,20091226.0,J960,NaN,4.0,84.0,...,84.0,>=80anos,F,NaN,NaN,J96,NaN,NaN,NaN,NaN
1,1,316240,19930726,3.0,20100109.0,20100113.0,A490,NaN,4.0,16.0,...,16.0,15a19,F,NaN,1.0,A49,NaN,NaN,NaN,NaN
2,1,316240,19470723,1.0,20100108.0,20100112.0,I500,NaN,4.0,62.0,...,62.0,60a64,M,NaN,2.0,I50,NaN,NaN,NaN,NaN
3,1,316240,19471102,1.0,20100110.0,20100115.0,I500,NaN,4.0,62.0,...,62.0,60a64,M,NaN,3.0,I50,NaN,NaN,NaN,NaN
4,1,316240,19880313,1.0,20100103.0,20100107.0,J158,NaN,4.0,21.0,...,21.0,20a24,M,NaN,4.0,J15,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12099951,1,317180,19400904,3.0,20191211.0,20191214.0,J449,0.0,4.0,79.0,...,79.0,75a79,F,Parda,NaN,J44,doencviaaer,bronquite,NaN,X
12099952,1,315750,20030421,1.0,20191227.0,20191230.0,J189,0.0,4.0,16.0,...,16.0,15a19,M,Parda,NaN,J18,NaN,NaN,NaN,NaN
12099953,1,317180,19851212,1.0,20191227.0,20191231.0,K746,0.0,4.0,34.0,...,34.0,30a34,M,Parda,NaN,K74,NaN,NaN,NaN,NaN
12099954,1,317180,20191129,3.0,20191203.0,20191205.0,P599,0.0,2.0,4.0,...,0.0,0a4,F,Parda,NaN,P59,NaN,NaN,NaN,NaN


In [19]:
sih.columns

Index(['IDENT', 'MUNIC_RES', 'NASC', 'SEXO', 'DT_INTER', 'DT_SAIDA',
       'DIAG_PRINC', 'DIAG_SECUN', 'COD_IDADE', 'IDADE', 'DIAS_PERM', 'MORTE',
       'CAR_INT', 'INSTRU', 'RACA_COR', 'data_inter', 'data_saida',
       'ano_inter', 'mes_inter', 'ano_saida', 'mes_saida', 'nascimento',
       'idade2', 'faixaetaria', 'sexo2', 'raca2', 'esc', 'causa3', 'agravo',
       'subgrupo', 'semana', 'capitulo'],
      dtype='object')

In [20]:
#obitos de agravos por faixa etaria municipio ano e mes
aux = sih[['MUNIC_RES', 'faixaetaria', 'ano_inter', 'mes_inter', 'agravo', 'IDADE']].groupby(['MUNIC_RES', 'faixaetaria', 'ano_inter', 'mes_inter', 'agravo'], as_index=False).count()
aux.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'inter_total']

#adiciona os agravos considerados como "subgrupos" na tabela
aux_sub = sih[['MUNIC_RES', 'faixaetaria', 'ano_inter', 'mes_inter', 'subgrupo', 'IDADE']].groupby(['MUNIC_RES', 'faixaetaria', 'ano_inter', 'mes_inter', 'subgrupo'], as_index=False).count()
aux_sub.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'inter_total']

aux = aux.append(aux_sub, ignore_index=True)
aux

,ID_Municipio,faixa_etaria,ano,mes,agravo,inter_total
0,110001,75a79,2019.0,10.0,doencviaaer,1
1,110002,50a54,2019.0,7.0,doenisquem,2
2,110002,65a69,2011.0,9.0,doencviaaer,1
3,110002,70a74,2018.0,9.0,doenisquem,1
4,110004,65a69,2014.0,5.0,doenisquem,2
...,...,...,...,...,...,...
1612344,530170,45a49,2017.0,5.0,iamrecor,1
1612345,530180,0a4,2017.0,10.0,bronquite,1
1612346,530180,55a59,2017.0,1.0,iamrecor,1
1612347,530180,55a59,2017.0,11.0,iamrecor,1


In [21]:
def gerar_inter(sih, sexo2, nome_inter):
    sih_aux = sih[sih['sexo2'] == sexo2]

    aux = sih_aux[['MUNIC_RES', 'faixaetaria', 'ano_inter', 'mes_inter', 'agravo', 'IDADE']].groupby(['MUNIC_RES', 'faixaetaria', 'ano_inter', 'mes_inter', 'agravo'], as_index=False).count()
    aux.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', f'inter_{nome_inter}']

    aux_sub = sih_aux[['MUNIC_RES', 'faixaetaria', 'ano_inter', 'mes_inter', 'subgrupo', 'IDADE']].groupby(['MUNIC_RES', 'faixaetaria', 'ano_inter', 'mes_inter', 'subgrupo'], as_index=False).count()
    aux_sub.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', f'inter_{nome_inter}']

    aux = aux.append(aux_sub, ignore_index=True)
    
    return aux

In [22]:
aux_h = gerar_inter(sih, 'M', "homem")
aux_m = gerar_inter(sih, 'F', "mulher")

In [23]:
aux = aux.merge(aux_h, how='left')
aux = aux.merge(aux_m, how='left')

In [24]:
aux

,ID_Municipio,faixa_etaria,ano,mes,agravo,inter_total,inter_homem,inter_mulher
0,110001,75a79,2019.0,10.0,doencviaaer,1,NaN,1.0
1,110002,50a54,2019.0,7.0,doenisquem,2,NaN,2.0
2,110002,65a69,2011.0,9.0,doencviaaer,1,NaN,1.0
3,110002,70a74,2018.0,9.0,doenisquem,1,1.0,NaN
4,110004,65a69,2014.0,5.0,doenisquem,2,2.0,NaN
...,...,...,...,...,...,...,...,...
1612344,530170,45a49,2017.0,5.0,iamrecor,1,NaN,1.0
1612345,530180,0a4,2017.0,10.0,bronquite,1,1.0,NaN
1612346,530180,55a59,2017.0,1.0,iamrecor,1,1.0,NaN
1612347,530180,55a59,2017.0,11.0,iamrecor,1,NaN,1.0


In [25]:
taxas = agregados.merge(aux, how='left')

taxas = taxas.fillna(0)
taxas = taxas.astype({'inter_total': 'int32',
                        'inter_homem': 'int32',
                        'inter_mulher': 'int32'})
taxas

,ID_Municipio,faixa_etaria,ano,mes,agravo,inter_total,inter_homem,inter_mulher
0,310020,0a4,2010,1,colera,0,0,0
1,310020,0a4,2010,1,diarreia,0,0,0
2,310020,0a4,2010,1,odip,2,2,0
3,310020,0a4,2010,1,tubresp,0,0,0
4,310020,0a4,2010,1,tbsistnerv,0,0,0
...,...,...,...,...,...,...,...,...
13871995,317120,>=80anos,2019,12,asma,1,0,1
13871996,317120,>=80anos,2019,12,broncoeq,0,0,0
13871997,317120,>=80anos,2019,12,infecpele,0,0,0
13871998,317120,>=80anos,2019,12,outraspele,0,0,0


In [26]:
def calcular_taxa_inter(taxa_inter, taxa_pop):
    return (taxa_inter / taxa_pop) * 100000

def calcular_inter_esp(taxa_pop_pad, taxa_inter):
    return taxa_pop_pad * (taxa_inter / 100000)

def calcular_taxa_inter_esp(taxa_inter_esp, taxa_pop_pad):
    return (taxa_inter_esp / taxa_pop_pad) * 100000

def calcular_taxas(taxas):
    labels = ['total', 'homem', 'mulher']
    for t in labels:
        taxas[f'taxa_inter_{t}']  = calcular_taxa_inter(taxas[f'inter_{t}'], taxas[f'pop_{t}'])
    
    for t in labels:
        taxas[f'inter_esp_{t}']  = calcular_inter_esp(taxas[f'pop_pad_{t}'], taxas[f'taxa_inter_{t}'])
    
    for t in labels:
        taxas[f'taxa_inter_esp_{t}']  = calcular_taxa_inter_esp(taxas[f'inter_esp_{t}'], taxas[f'pop_pad_{t}'])
    
    return taxas

In [27]:
taxas = df.merge(taxas, how='inner')

taxas = calcular_taxas(taxas)

In [28]:
taxas['grupo'] = np.nan

taxas.loc[(taxas['Município'].isin(cidades_selecionadas)), 'grupo'] = 1
taxas.loc[(taxas['Município'].isin(cidades_limitrofes)), 'grupo'] = 2
taxas.loc[(taxas['Município'].isin(cidades_prox_limitrofes)), 'grupo'] = 3

taxas = taxas[~taxas['grupo'].isnull()]

In [29]:
taxas

,Município,ID_Municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,...,taxa_inter_total,taxa_inter_homem,taxa_inter_mulher,inter_esp_total,inter_esp_homem,inter_esp_mulher,taxa_inter_esp_total,taxa_inter_esp_homem,taxa_inter_esp_mulher,grupo
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
2,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,145.243282,284.900285,0.000000,1944.158315,1950.509972,0.000000,145.243282,284.900285,0.000000,2.0
3,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
4,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13871995,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,66.357001,0.000000,102.145046,221.985401,0.000000,209.500511,66.357001,0.000000,102.145046,3.0
13871996,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
13871997,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
13871998,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0


In [30]:
taxas.columns

Index(['Município', 'ID_Municipio', 'ano', 'faixa_etaria', 'pop_total',
       'pop_pad_total', 'pop_homem', 'pop_pad_homem', 'pop_mulher',
       'pop_pad_mulher', 'mes', 'agravo', 'inter_total', 'inter_homem',
       'inter_mulher', 'taxa_inter_total', 'taxa_inter_homem',
       'taxa_inter_mulher', 'inter_esp_total', 'inter_esp_homem',
       'inter_esp_mulher', 'taxa_inter_esp_total', 'taxa_inter_esp_homem',
       'taxa_inter_esp_mulher', 'grupo'],
      dtype='object')

In [31]:
taxas.columns = ['municipio', 'id_municipio', 'ano', 'faixa_etaria', 'pop_total',
       'pop_pad_total', 'pop_homem', 'pop_pad_homem', 'pop_mulher',
       'pop_pad_mulher', 'mes', 'agravo', 'inter_total',
       'inter_homem', 'inter_mulher', 'taxa_inter_total',
       'taxa_inter_homem', 'taxa_inter_mulher', 'inter_esp_total',
       'inter_esp_homem', 'inter_esp_mulher', 'taxa_inter_esp_total',
       'taxa_inter_esp_homem', 'taxa_inter_esp_mulher', 'grupo']

taxas

,municipio,id_municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,...,taxa_inter_total,taxa_inter_homem,taxa_inter_mulher,inter_esp_total,inter_esp_homem,inter_esp_mulher,taxa_inter_esp_total,taxa_inter_esp_homem,taxa_inter_esp_mulher,grupo
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
2,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,145.243282,284.900285,0.000000,1944.158315,1950.509972,0.000000,145.243282,284.900285,0.000000,2.0
3,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
4,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13871995,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,66.357001,0.000000,102.145046,221.985401,0.000000,209.500511,66.357001,0.000000,102.145046,3.0
13871996,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
13871997,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
13871998,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0


In [32]:
taxas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13872000 entries, 0 to 13871999
Data columns (total 25 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   municipio              object 
 1   id_municipio           int64  
 2   ano                    int64  
 3   faixa_etaria           object 
 4   pop_total              int64  
 5   pop_pad_total          int64  
 6   pop_homem              int64  
 7   pop_pad_homem          int64  
 8   pop_mulher             int64  
 9   pop_pad_mulher         int64  
 10  mes                    int64  
 11  agravo                 object 
 12  inter_total            int32  
 13  inter_homem            int32  
 14  inter_mulher           int32  
 15  taxa_inter_total       float64
 16  taxa_inter_homem       float64
 17  taxa_inter_mulher      float64
 18  inter_esp_total        float64
 19  inter_esp_homem        float64
 20  inter_esp_mulher       float64
 21  taxa_inter_esp_total   float64
 22  taxa_inter_esp_h

In [33]:
taxas.to_csv('pre_taxa_sih.csv', index=False)